## 21i2478 Muhammad Tayyab Sohail

### Artificial Intelligence Project

### Sec B 

### TimeTable Through Genetic Algorithm

In [8]:
import pandas as pd
import random

subjects = {
    "Artificial Intelligence LAB": "0001",
    "Artificial Intelligence": "0010",
    "PDC": "0011",
    "Numerical Computing": "0100",
    "Software Engineering": "0101",
    "Web Programming": "0110"
}


teachers = {
    "Amina Ashfaq": "0001",  # same as AI LAB
    "Usama Imtiaz": "0010",  # same as AI 
    "Muhammad Aleem": "0011",# same as PDC 
    "Imran Ashraf": "0100",  # same as NUMERICAL 
    "Javeria Zia": "0101",   # same as SE 
    "Aqib Rehman": "0110"    # Asame as WEB 
}

sections = {
    "A": {"strength": 90},
    "B": {"strength": 50},
    "C": {"strength": 70}
}

rooms = {
    "A-301": "0001",
    "A-302": "0010",
    "A-303": "0011",
    "A-304": "0100",
    "A-305": "0101",
    "A-306": "0110",
    "A-307": "0111",
    "A-308": "1000",
    "A-309": "1001",
    "A-310": "1010",
    "A-311": "1011",
    "A-312": "1100"
}

# Define timings in binary format
timings_class = {
    "8:30 - 9:50": "0001",
    "10:00 - 11:20": "0010",
    "11:30 - 12:50": "0011",
    "13:00 - 14:20": "0100"  
}

timings_lab = {
    "14:30 - 17:30": "0001"
}

days = {
    1: "0001",
    2: "0010",
    3: "0011",
    4: "0100",
    5: "0101"
}

def generate_timeslot(day):
    return random.choice(list(timings_class.keys()))

def create_chromosome(subjects, teachers, sections, rooms, timings_class, timings_lab, days):
    all_timetable_chromosomes = []
    for section_id in sections:
        section_strength = random.choice(["90", "50", "70"])
        # Generate two unique days for theory classes
        theory_days = random.sample(list(days.values()), 2)
        for subject in subjects:
            teacher = None
            if subject == "Artificial Intelligence LAB":
                teacher = "Amina Ashfaq"  
                theory_lab = 1  # lab class
            else:
                teacher = [key for key, value in teachers.items() if value == subjects[subject]][0]  #teacher per subject
                theory_lab = 0  # theory class
            for chosen_day in theory_days:
                timeslot = generate_timeslot(chosen_day)
                room = random.choice(list(rooms.keys()))  # choose classroom randomly

                # Removing wrong values
                if not (subject == "Artificial Intelligence LAB" and chosen_day == "0101" and room == "A-311" and timeslot in ["0010", "0011"]):
                    chromosome = {
                        "Day": chosen_day,
                        "Subject": subjects[subject],
                        "Theory/Lab": "00000010" if theory_lab == 0 else "00000001",
                        "Section": "0000000" + str(ord(section_id) - ord('A') + 1),
                        "Section Strength": "{:08b}".format(int(section_strength)),
                        "Professor": teachers[teacher],
                        "Timeslot": timings_class[timeslot] if theory_lab == 0 else list(timings_lab.values())[0],
                        "Room": rooms[room],
                        "Room Size": "00000001" if random.randint(0, 1) == 0 else "00000010"
                    }

                    all_timetable_chromosomes.append(chromosome)

    # Lab per section
    for section_id in sections:
        lab_day = random.choice(list(days.values()))  
        lab_timeslot = list(timings_lab.keys())[0]  
        lab_room = random.choice(list(rooms.keys()))  

        chromosome = {
            "Day": lab_day,
            "Subject": subjects["Artificial Intelligence LAB"],
            "Theory/Lab": "00000001",
            "Section": "0000000" + str(ord(section_id) - ord('A') + 1),
            "Section Strength": "{:08b}".format(int(section_strength)),
            "Professor": teachers["Amina Ashfaq"],
            "Timeslot": timings_lab[lab_timeslot],
            "Room": rooms[lab_room],
            "Room Size": "00000001" if random.randint(0, 1) == 0 else "00000010"  
        }
        all_timetable_chromosomes.append(chromosome)

    #Print
    timetable_df = pd.DataFrame(all_timetable_chromosomes)

    return timetable_df


def display_timetable_with_english_names(df):
    subject_mapping = {v: k for k, v in subjects.items()}
    lab_mapping = {"00000001": "Lab", "00000010": "Theory"}
    room_size_mapping = {"00000001": "Classroom", "00000010": "Large Hall"}
    section_mapping = {"0001": "A", "0010": "B", "0011": "C"}
    professor_mapping = {v: k for k, v in teachers.items()}
    timeslot_mapping = {v: k for k, v in timings_class.items()}
    room_mapping = {v: k for k, v in rooms.items()}

    df['Subject'] = df['Subject'].map(subject_mapping)
    df['Theory/Lab'] = df['Theory/Lab'].map(lab_mapping)
    df['Room Size'] = df['Room Size'].map(room_size_mapping)
    df['Professor'] = df['Professor'].map(professor_mapping)
    df['Timeslot'] = df['Timeslot'].map(timeslot_mapping)
    df['Room'] = df['Room'].map(room_mapping)

    # Drop the 'Section Strength' column
    df.drop(columns=['Section Strength'], inplace=True)

    print(df.to_string(index=False))



def display_timetable_with_names_and_binary(df):
    print(" ")
    print("Timetable with Binary Values:")
    print(df.to_string(index=False))


def fitness_function(df):
    conflicts = 0

    for index, row in df.iterrows():
        # Check if a teacher is taking multiple classes at the same time
        teacher = row['Professor']
        day = row['Day']
        timeslot = row['Timeslot']
        if len(df[(df['Professor'] == teacher) & (df['Day'] == day) & (df['Timeslot'] == timeslot)]) > 1:
            conflicts += 1

        # Check if a room is assigned to multiple classes at the same time
        room = row['Room']
        if len(df[(df['Room'] == room) & (df['Day'] == day) & (df['Timeslot'] == timeslot)]) > 1:
            conflicts += 1

        # Check if the strength of the room is suitable
        room_size = row['Room Size']
        section = row['Section']
        if section in sections:
            strength = sections[section]['strength']
            if room_size == '00000010':  # Large Hall
                if strength < 60:        #  strength <  60
                    conflicts += 1
            elif room_size == '00000001':# Classroom
                if strength > 60:        # strength > 60
                    conflicts += 1

        # Check if the same subject is scheduled consecutively for the same section
        prev_row = df.iloc[index - 1] if index > 0 else None
        if prev_row is not None and prev_row['Subject'] == row['Subject'] and prev_row['Section'] == row['Section']:
            prev_day_index = list(days.values()).index(prev_row['Day'])
            curr_day_index = list(days.values()).index(row['Day'])
            if curr_day_index - prev_day_index == 1:  # Consecutive days
                conflicts += 1

        # Check if one section has only one class at one time
        if len(df[(df['Section'] == row['Section']) & (df['Day'] == day) & (df['Timeslot'] == timeslot)]) > 1:
            conflicts += 1

    return conflicts

def generate_and_evaluate_chromosomes(num_chromosomes):
    all_chromosomes = []

    for i in range(num_chromosomes):
       # print(f"Chromosome {i + 1}:")
        chromosome = create_chromosome(subjects, teachers, sections, rooms, timings_class, timings_lab, days)
        #print("Timetable with Binary Values:")
        #display_timetable_with_names_and_binary(chromosome)
        #print("\nTimetable with English Names:")
        #display_timetable_with_english_names(chromosome)
        conflicts = fitness_function(chromosome)
        print("Total conflicts in the timetable:", conflicts)
        print("\n")
        all_chromosomes.append((chromosome, conflicts))
    
    return all_chromosomes

def select_parents(population):
    sorted_population = sorted(population, key=lambda x: x[1]) # Sort on fitness
    #print("Parents",sorted_population[0],sorted_population[1])
    return sorted_population[:2]

def crossover(parent1, parent2):
   
    # Initialize children as copies of parents
    child1 = parent1.copy()
    child2 = parent2.copy()

    # Select a random day for crossover
    crossover_day = random.choice(['0001', '0010', '0011', '0100', '0101'])
    crossover_day=str( crossover_day)
    

    # Extract timetable information for the selected day from parents
    parent1_day_info = parent1[parent1['Day'] == crossover_day].copy()
    parent2_day_info = parent2[parent2['Day'] == crossover_day].copy()
  
    timeslot_list = parent1_day_info['Timeslot'].tolist()
    random.shuffle(timeslot_list)
    parent1_day_info['Timeslot'] = timeslot_list
    timeslot_list = parent2_day_info['Timeslot'].tolist()
    random.shuffle(timeslot_list)
    parent2_day_info['Timeslot'] = timeslot_list

    # Update children with shuffled timetable entries for the selected day
    child1[child1['Day'] == crossover_day] = parent1_day_info.values
    child2[child2['Day'] == crossover_day] = parent2_day_info.values
    
    # # Print conflicts in parents and children
    # print("Conflicts in Parent 1:")
    # print(fitness_function(parent1))
    # print("\nConflicts in Parent 2:")
    # print(fitness_function(parent2))
    # print("\nConflicts in Child 1:")
    # print(fitness_function(child1))
    # print("\nConflicts in Child 2:")
    # print(fitness_function(child2))

    return child1, child2


def mutation(child):
     
    for day in range(1, 6):
        day= bin(day)[2:].zfill(4)

        mutation_chance = random.randint(1, 100)
        if mutation_chance > 50:  
            
            day_entries = child[child['Day'] == day]
           
          
            if len(day_entries) > 1:
                
                idx1, idx2 = random.sample(day_entries.index.tolist(), 2)
                temp_row = child.loc[idx1].copy()
                child.loc[idx1] = child.loc[idx2]
                child.loc[idx2] = temp_row
    # print("After Mutation",fitness_function(child))
    return child


def evolve(selected_parents):
    children = []                                      
    parent1, parent2 = random.sample(selected_parents, 2)
    child1, child2 = crossover(parent1[0], parent2[0])
    child1 = mutation(child1)
    child2 = mutation(child2)
    child1=(child1,fitness_function(child1))
    child2=(child2,fitness_function(child2))
    
    
    
    return child1,child2

def replace_worst(population, child1,child2):
   
    population.sort(key=lambda x: x[1])
    
    population =  population[:-2]
   
    population.append(child1)
    population.append(child2)
  
    return population


def find_best_chromosome(population):
    best_chromosome = None
    least_conflicts = float('inf')

    for chromosome, conflicts in population:
        if conflicts < least_conflicts:
            best_chromosome = chromosome
            least_conflicts = conflicts

    return best_chromosome, least_conflicts

population = generate_and_evaluate_chromosomes(10)         #evaluate 10 chromosomes

#sorted_population = sorted(population, key=lambda x: x[1]) # Sort on fitness

print("Initial Population Conflicts:")
for individual, conflicts in population:
    print("Conflicts:", conflicts)

print(" ")
print(" -------------------------------------------------- ")
print(" ")

num_iterations = 500

for iteration in range(num_iterations):
    selected_parents = select_parents(population)
    child1,child2 = evolve(selected_parents)     
    population = replace_worst(population,child1,child2)
 
    # print("\n" + "-"*50 + "\n")

print("Final Population Conflicts:")
min=100000

for individual, conflicts in population:
    if(conflicts<min):
        min=conflicts
        mintable=individual     
    print("Conflicts:", conflicts)
print("Best Fitness:\n",min)
print("Best Timetable:\n",mintable)

best_timetable = mintable  # Assuming 'mintable' contains the best timetable from the final population
display_timetable_with_english_names(best_timetable)


Total conflicts in the timetable: 49


Total conflicts in the timetable: 43


Total conflicts in the timetable: 57


Total conflicts in the timetable: 52


Total conflicts in the timetable: 55


Total conflicts in the timetable: 58


Total conflicts in the timetable: 47


Total conflicts in the timetable: 54


Total conflicts in the timetable: 43


Total conflicts in the timetable: 45


Initial Population Conflicts:
Conflicts: 49
Conflicts: 43
Conflicts: 57
Conflicts: 52
Conflicts: 55
Conflicts: 58
Conflicts: 47
Conflicts: 54
Conflicts: 43
Conflicts: 45
 
 -------------------------------------------------- 
 
Final Population Conflicts:
Conflicts: 29
Conflicts: 29
Conflicts: 29
Conflicts: 29
Conflicts: 29
Conflicts: 29
Conflicts: 29
Conflicts: 29
Conflicts: 38
Conflicts: 36
Best Fitness:
 29
Best Timetable:
      Day Subject Theory/Lab   Section Section Strength Professor Timeslot  \
0   0011    0011   00000010  00000002         01011010      0011     0001   
1   0010    0001   0000000